https://medium.com/@curiousily/tensorflow-for-hackers-part-ii-building-simple-neural-network-2d6779d2f91b

In [ ]:
import pandas as pd
from math import floor
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

In [2]:
data1 = pd.read_csv('premier_14_15.csv')
data2 = pd.read_csv('premier_15_16.csv')
data3 = pd.read_csv('premier_16_17.csv')
data4 = pd.read_csv('premier_17_18.csv')

dataCon = [data1,data2,data3,data4]
data = pd.concat(dataCon)

In [3]:
b = np.random.uniform(low=-0.01,high=0.01,size=(3,3))
a = pd.DataFrame(b,columns=['a','b','c'])
a

,a,b,c
0,-0.004935,0.002548,-0.004550
1,0.003464,0.005043,-0.004403
2,0.009187,0.003748,0.008686


In [4]:
def normalization(raw_data):
    for col_num in range(raw_data.shape[1]):
        if raw_data.iloc[:,col_num].dtype == np.float or raw_data.iloc[:,col_num].dtype == np.int:
            raw_data.iloc[:,col_num] = (raw_data.iloc[:,col_num] - raw_data.iloc[:,col_num].mean()) / (raw_data.iloc[:,col_num].max() - raw_data.iloc[:,col_num].min())
    return raw_data

In [5]:
def embedding_matrix(column):
    labels = []
    embeddings = np.array([])
    num_of_uniques = len(np.unique(column))
    for i in range(num_of_uniques):
        if embeddings.size == 0:
            embeddings = np.random.uniform(low=-0.01,high=0.01,size=(min(50,(num_of_uniques+1)//2),1))
        else:
            embeddings = np.append(embeddings,np.random.uniform(low=-0.01,high=0.01,size=(min(50,(num_of_uniques+1)//2),1)),axis=1)
        labels.append(np.unique(column)[i])
    print("embeddings.shape:",embeddings.shape)
    print("labels.shape:",len(labels))
    return pd.DataFrame(data=embeddings,columns=labels)

In [6]:
em = embedding_matrix(data['HomeTeam'])
em.head()

embeddings.shape: (13, 26)
labels.shape: 26


,Arsenal,Aston Villa,Bournemouth,Brighton,Burnley,Chelsea,Crystal Palace,Everton,Huddersfield,Hull,...,Norwich,QPR,Southampton,Stoke,Sunderland,Swansea,Tottenham,Watford,West Brom,West Ham
0,-0.004364,-0.004535,0.000449,-0.006519,0.003545,0.004314,-0.003235,-0.007671,0.007767,0.004150,...,0.000097,-0.005498,0.006047,0.004349,0.005006,-0.003944,-0.007810,-0.002923,-0.005785,0.005636
1,0.007751,-0.002451,-0.003228,-0.000107,0.000216,-0.001690,-0.009091,-0.002545,0.001061,0.001960,...,0.000992,0.004493,0.004126,0.005838,0.009875,0.002653,0.002558,0.001683,0.006857,0.007785
2,-0.007168,0.005717,0.005881,-0.008429,0.003524,-0.005608,0.004286,-0.006886,-0.001122,-0.003192,...,0.005820,0.008808,0.000491,0.009841,-0.003581,0.002147,-0.007667,0.002156,0.001814,0.008561
3,-0.008783,-0.007929,0.000131,-0.001313,0.004686,0.000065,0.005990,0.005486,-0.008769,0.004907,...,0.009400,-0.003534,0.001548,-0.004449,0.007493,-0.003694,0.004089,0.009684,0.005499,0.000218
4,0.004476,-0.001079,-0.003395,0.002110,-0.009380,0.006157,-0.005553,0.004904,0.003092,0.009887,...,-0.009138,0.003990,-0.002369,-0.004660,0.005852,0.004552,0.007358,-0.004808,0.007657,0.008028


In [7]:
#x_data = data.drop(['FTR','FTAG','FTHG'],1)
data = normalization(data)
x_data = np.column_stack((np.transpose(em[data['HomeTeam']].values),
                          np.transpose(em[data['AwayTeam']].values),
                          data['B365H'][:,None],
                          data['B365D'][:,None],
                          data['B365A'][:,None]))
y_data = data['FTR']

y_data = pd.get_dummies(y_data)

#differences = data['FTHG']-data['FTAG']

In [ ]:
#x_data = data.drop(['FTR','FTAG','FTHG'],1)
x_data = np.column_stack((data['B365H'][:,None],
                          data['B365D'][:,None],
                          data['B365A'][:,None]))
y_data = data['FTR']

y_data = pd.get_dummies(y_data)

#differences = data['FTHG']-data['FTAG']

In [8]:
x_data.shape

(1520, 29)

In [9]:
# Two first games
x_data[:,0:2]

array([[-4.36357344e-03,  7.75083481e-03],
       [-5.38420681e-06,  7.01560683e-03],
       [-4.75052582e-03,  5.27273581e-03],
       ...,
       [-3.94373079e-03,  2.65290614e-03],
       [-7.81034267e-03,  2.55836017e-03],
       [ 5.63643997e-03,  7.78462827e-03]])

In [10]:
y_data.shape

(1520, 3)

In [11]:
y_data.head()

,A,D,H
0,0,0,1
1,0,1,0
2,1,0,0
3,1,0,0
4,1,0,0


In [12]:
train_size = 0.9
valid_size = 0.3

train_cnt = floor(x_data.shape[0] * train_size)

x_train = x_data[0:train_cnt]
y_train = y_data.iloc[0:train_cnt].values

valid_cnt = floor((x_data.shape[0] - train_cnt) * valid_size)

x_valid = x_data[train_cnt:train_cnt+valid_cnt]
y_valid = y_data.iloc[train_cnt:train_cnt+valid_cnt].values

x_test = x_data[train_cnt+valid_cnt:]
y_test = y_data.iloc[train_cnt+valid_cnt:]

print("x_train:",x_train.shape)
print("x_valid:",x_valid.shape)
print("x_test:",x_test.shape)

x_train: (1368, 29)
x_valid: (45, 29)
x_test: (107, 29)


In [13]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

In [14]:
# Parameters
ALPHA = 1e-3
num_epochs = 50
batch_size = 128
display_step = 5

In [15]:
# Network Parameters
num_input = x_data.shape[1]
num_classes = y_data.shape[1]
num_hidden_1 = 50
num_hidden_2 = 50
KEEP_PROB_STA = 0.9

In [16]:
def neural_network(x,weights,biases,keep_prob):
    layer_1 = tf.add(tf.matmul(x,weights['w1']),biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1,keep_prob)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['w2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2,keep_prob)
    
    layer_out = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return layer_out

In [17]:
# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([num_input,num_hidden_1])),
    'w2': tf.Variable(tf.random_normal([num_hidden_1,num_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_hidden_2, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

keep_prob = tf.placeholder("float")

In [18]:
predictions = neural_network(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE_STA).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [19]:
costs = []
lrn_rate_sizes = []

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: KEEP_PROBABILITY
                            })
            avg_cost += c / total_batch
            
        if epoch % display_step == 0:
            print("Train: Epoch:", '%04d' % (epoch+display_step), "cost=", "{:.9f}".format(avg_cost))
            _, valid_cost = sess.run([optimizer, cost], feed_dict={x: x_valid, y: y_valid, keep_prob: 1})
            print("Valid: Epoch:", '%04d' % (epoch+display_step), "cost=", "{:.9f}".format(valid_cost))
            
            correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
            print("______________________________________")
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

Train: Epoch: 0005 cost= 10.892515278
Valid: Epoch: 0005 cost= 4.052656174
Accuracy: 0.42990655
______________________________________
Train: Epoch: 0010 cost= 8.005325937
Valid: Epoch: 0010 cost= 3.773820639
Accuracy: 0.5607477
______________________________________
Train: Epoch: 0015 cost= 7.297330284
Valid: Epoch: 0015 cost= 3.338582993
Accuracy: 0.5794392
______________________________________
Train: Epoch: 0020 cost= 6.497834110
Valid: Epoch: 0020 cost= 3.189132452
Accuracy: 0.5607477
______________________________________
Train: Epoch: 0025 cost= 5.821731710
Valid: Epoch: 0025 cost= 2.694743872
Accuracy: 0.5607477
______________________________________
Train: Epoch: 0030 cost= 5.473375702
Valid: Epoch: 0030 cost= 2.208714724
Accuracy: 0.5607477
______________________________________
Train: Epoch: 0035 cost= 4.468893528
Valid: Epoch: 0035 cost= 2.375280619
Accuracy: 0.55140185
______________________________________
Train: Epoch: 0040 cost= 4.191141105
Valid: Epoch: 0040 cost= 1.99